# 제품 이상여부 판별 프로젝트

## 1. 데이터 불러오기

### 필수 라이브러리

In [1]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 엑셀 파일을 읽는 함수

읽어오는 속도가 느린 엑셀 파일을 위해 csv 파일로 변환하여 저장해 두고 사용합니다.

In [2]:
def read_excel_file(file_path: str, header: int = None) -> pd.DataFrame:
    csv_file = file_path.replace(".xlsx", ".csv")

    if not os.path.exists(csv_file):
        print("Converting excel to csv...")
        if header:
            df = pd.read_excel(file_path, header=header)
        else:
            df = pd.read_excel(file_path)

        df.to_csv(csv_file, index=False)
        print(f"  {file_path} -> {csv_file}")
        return df
    else:
        print(f"  Reading {csv_file}")
        return pd.read_csv(csv_file, low_memory=False)

### 엑셀 파일들 읽어오기

In [3]:
ROOT_DIR = "data"
RANDOM_STATE = 110

X_Dam = read_excel_file(os.path.join(ROOT_DIR, "Dam dispensing.xlsx"), header=1)

X_AutoClave = read_excel_file(
    os.path.join(ROOT_DIR, "Auto clave.xlsx"), header=1
)

X_Fill1 = read_excel_file(
    os.path.join(ROOT_DIR, "Fill1 dispensing.xlsx"), header=1
)

X_Fill2 = read_excel_file(
    os.path.join(ROOT_DIR, "Fill2 dispensing.xlsx"), header=1
)

y = pd.read_csv(os.path.join(ROOT_DIR, "train_y.csv"))

  Reading data\Dam dispensing.csv
  Reading data\Auto clave.csv
  Reading data\Fill1 dispensing.csv
  Reading data\Fill2 dispensing.csv


### 데이터 병합

x 데이터 병합

In [4]:
# Rename columns
X_Dam.columns = [i + " - Dam" for i in X_Dam.columns]
X_AutoClave.columns = [i + " - AutoClave" for i in X_AutoClave.columns]
X_Fill1.columns = [i + " - Fill1" for i in X_Fill1.columns]
X_Fill2.columns = [i + " - Fill2" for i in X_Fill2.columns]
X_Dam = X_Dam.rename(columns={"Set ID - Dam": "Set ID"})
X_AutoClave = X_AutoClave.rename(columns={"Set ID - AutoClave": "Set ID"})
X_Fill1 = X_Fill1.rename(columns={"Set ID - Fill1": "Set ID"})
X_Fill2 = X_Fill2.rename(columns={"Set ID - Fill2": "Set ID"})

# Merge X
X = pd.merge(X_Dam, X_AutoClave, on="Set ID")
X = pd.merge(X, X_Fill1, on="Set ID")
X = pd.merge(X, X_Fill2, on="Set ID")
X = X.drop(X[X.duplicated(subset="Set ID")].index).reset_index(drop=True)
X

,Wip Line - Dam,Process Desc. - Dam,Equipment - Dam,Model.Suffix - Dam,Workorder - Dam,LOT ID - Dam,Set ID,Box ID - Dam,Collect Date - Dam,Insp. Seq No. - Dam,...,Collect Result.37 - Fill2,Unit Time.37 - Fill2,Judge Value.37 - Fill2,Collect Result.38 - Fill2,Unit Time.38 - Fill2,Judge Value.38 - Fill2,Collect Result.39 - Fill2,Unit Time.39 - Fill2,Judge Value.39 - Fill2,Unnamed: 131 - Fill2
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000002,OP753345013050000002,NaN,2023-05-04 08:57:23,1,...,1,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000003,OP753345013050000003,NaN,2023-05-04 09:11:35,1,...,2,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
2,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000004,OP753345013050000004,NaN,2023-05-04 09:13:19,1,...,3,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
3,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000005,OP753345013050000005,NaN,2023-05-04 09:15:24,1,...,4,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000006,OP753345013050000006,NaN,2023-05-04 09:17:27,1,...,5,NaN,NaN,1,NaN,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57862,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002685,OP753345054040002685,NaN,2024-04-28 18:30:42,1,...,11,NaN,NaN,435,NaN,NaN,1,NaN,NaN,0.0
57863,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002686,OP753345054040002686,NaN,2024-04-28 18:31:43,1,...,5,NaN,NaN,436,NaN,NaN,1,NaN,NaN,0.0
57864,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002687,OP753345054040002687,NaN,2024-04-28 18:32:49,1,...,12,NaN,NaN,437,NaN,NaN,1,NaN,NaN,0.0
57865,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002688,OP753345054040002688,NaN,2024-04-28 18:33:51,1,...,6,NaN,NaN,438,NaN,NaN,1,NaN,NaN,0.0


x 데이터와 y 데이터 병합

In [5]:
# Merge X and y
df_merged = pd.merge(X, y, "inner", on="Set ID")

# Drop columns with more than half of the values missing
drop_cols = []
for column in df_merged.columns:
    if (df_merged[column].notnull().sum() // 2) < df_merged[
        column
    ].isnull().sum():
        drop_cols.append(column)
    if column == 'LOT ID - Dam' or column == 'Set ID' or column == 'Collect Date - Dam':
        continue
    elif len(df_merged[column].unique()) == df_merged.shape[0]:
        drop_cols.append(column)
    elif len(df_merged[column].unique()) == 1:
        drop_cols.append(column)
    
df_merged = df_merged.drop(drop_cols, axis=1)

# Drop Lot ID
# df_merged = df_merged.drop("LOT ID - Dam", axis=1)
df_merged

,Equipment - Dam,Model.Suffix - Dam,Workorder - Dam,LOT ID - Dam,Set ID,Collect Date - Dam,Collect Result - Dam,Collect Result.1 - Dam,Collect Result.2 - Dam,Collect Result.3 - Dam,...,Collect Result.31 - Fill2,Collect Result.32 - Fill2,Collect Result.33 - Fill2,Collect Result.34 - Fill2,Collect Result.35 - Fill2,Collect Result.36 - Fill2,Collect Result.37 - Fill2,Collect Result.38 - Fill2,Collect Result.39 - Fill2,target
0,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000003,OP753345013050000003,2023-05-04 09:11:35,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.200,5.0,2,1,0,AbNormal
1,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000004,OP753345013050000004,2023-05-04 09:13:19,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.400,6.0,3,1,0,AbNormal
2,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000007,OP753345013050000007,2023-05-04 09:19:31,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.600,1.0,6,1,0,AbNormal
3,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000011,OP753345013050000011,2023-05-04 11:19:55,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.200,4.0,18,1,0,AbNormal
4,Dam dispenser #1,AJX75334501,3F1X5847-2,OP753345013050000024,OP753345013050000024,2023-05-05 09:35:48,240.0,2.5,-90,70,...,91.8,270.0,50,85,18.100,6.0,12,1,0,AbNormal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002680,OP753345054040002680,2024-04-28 18:25:20,240.0,2.5,-90,100,...,50.0,91.8,270,50,114.612,18.8,2,430,1,Normal
40502,Dam dispenser #2,AJX75334505,4F1XB738-1,OP753345054040002681,OP753345054040002681,2024-04-28 18:26:27,1000.0,12.5,90,100,...,50.0,91.8,270,50,114.612,19.0,9,431,1,Normal
40503,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002682,OP753345054040002682,2024-04-28 18:27:27,240.0,2.5,-90,100,...,50.0,91.8,270,50,114.612,19.3,3,432,1,Normal
40504,Dam dispenser #1,AJX75334505,4F1XB738-1,OP753345054040002684,OP753345054040002684,2024-04-28 18:29:36,240.0,2.5,-90,100,...,50.0,91.8,270,50,114.612,19.6,4,434,1,Normal


In [6]:
# df_merged[col].unique()

In [7]:
# unique_values = {col: df_merged[col].unique() for col in df_merged.columns}

# drop_cols = []

# for col, values in unique_values.items():
#     if len(values) == df_merged.shape[0]:
#         # print(col)
#         drop_cols.append(col)
# # print('='*10)
# for col, values in unique_values.items():
#     if len(values) == 1:
#         # print(col)
#         drop_cols.append(col)

# drop_cols

In [8]:
# df_merged = df_merged.drop(drop_cols, axis=1)
# df_merged

### 언더 샘플링

데이타 불균형을 해결하기 위해 언더 샘플링을 진행합니다.

In [9]:
normal_ratio = 1.5  # 1.0 means 1:1 ratio

df_normal = df_merged[df_merged["target"] == "Normal"]
df_abnormal = df_merged[df_merged["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(
    n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE
)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
Normal      3525
AbNormal    2350
Name: count, dtype: int64

In [35]:
df_concat.to_csv('df_concat.csv')

In [36]:
[print(col) for col in df_concat.columns]

Equipment - Dam
Model.Suffix - Dam
Workorder - Dam
LOT ID - Dam
Set ID
Collect Date - Dam
Collect Result - Dam
Collect Result.1 - Dam
Collect Result.2 - Dam
Collect Result.3 - Dam
Collect Result.7 - Dam
Collect Result.9 - Dam
Collect Result.10 - Dam
Collect Result.11 - Dam
Collect Result.12 - Dam
Collect Result.13 - Dam
Collect Result.14 - Dam
Collect Result.15 - Dam
Collect Result.16 - Dam
Collect Result.17 - Dam
Collect Result.18 - Dam
Collect Result.19 - Dam
Collect Result.20 - Dam
Collect Result.21 - Dam
Collect Result.22 - Dam
Collect Result.23 - Dam
Collect Result.24 - Dam
Collect Result.25 - Dam
Collect Result.26 - Dam
Collect Result.27 - Dam
Collect Result.28 - Dam
Collect Result.29 - Dam
Collect Result.30 - Dam
Collect Result.31 - Dam
Collect Result.32 - Dam
Collect Result.33 - Dam
Collect Result.34 - Dam
Collect Result.35 - Dam
Collect Result.36 - Dam
Collect Result.37 - Dam
Collect Result.38 - Dam
Collect Result.39 - Dam
Collect Result.40 - Dam
Collect Result.41 - Dam
Collec

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [37]:
[print(col,'\n',df_concat[col].unique()) for col in df_concat.columns]

Equipment - Dam 
 ['Dam dispenser #1' 'Dam dispenser #2']
Model.Suffix - Dam 
 ['AJX75334501' 'AJX75334503' 'AJX75334502' 'AJX75334505' 'AJX75334506'
 'AJX75334507' 'AJX75334508']
Workorder - Dam 
 ['3F1X5847-2' '3F1X9643-1' '3F1X9644-1' '3F1XB560-1' '3F1XC376-1'
 '3F1XC414-1' '3F1XA351-1' '3FPM0081-1' '3FPM0085-1' '3GPM0070-1'
 '3G1XA417-1' '3F1X9648-1' '3F1XA350-1' '3F1XC600-1' '3G1XA482-1'
 '3G1XA481-1' '3G1XA484-2' '3G1XA483-1' '3G1XA484-1' '3F1XC781-1'
 '3G1XA502-1' '3G1XA503-2' '3G1XB044-1' '3F1X9648-2' '3G1XA503-1'
 '3F1XC414-2' '3G1X8290-1' '3G1XA501-1' '3G1X8646-1' '3G1X8646-2'
 '3G1X9014-1' '3G1X9016-2' '3G1X9016-1' '3G1XB177-1' '3G1XB159-1'
 '3G1X4501-1' '3G1XB234-1' '3G1X8296-1' '3G1XB416-1' '3G1X8295-1'
 '3G1XB176-1' '3G1XA384-1' '3G1XB278-1' '3G1X8298-2' '3G1XB045-1'
 '3FPXX064-0003' '3G1X9017-1' '3G1XB995-1' '3G1XB994-1' '3G1XB948-2'
 '3G1XB948-1' '3G1XB952-1' '3G1XA615-2' '3G1X4502-2' '3G1X8298-1'
 '3G1X8297-2' '3G1X8300-1' '3G1XC495-1' '3G1XC496-2' '3G1X8293-1'
 '3G1X8

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

### 데이터 분할

In [10]:
df_concat = df_concat.sort_values(by=["Collect Date - Dam"])
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.2,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)


def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(
        f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}"
        + f" ratio: {num_abnormal/num_normal}"
    )


# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 2820, AbNormal: 1880 ratio: 0.6666666666666666
  Total: Normal: 705, AbNormal: 470 ratio: 0.6666666666666666


## 3. 모델 학습

### 모델 정의 

In [11]:
# model = RandomForestClassifier(random_state=RANDOM_STATE)

### 모델 학습

In [12]:
# features = []

# for col in df_train.columns:
#     try:
#         df_train[col] = df_train[col].astype(int)
#         features.append(col)
#     except:
#         continue

# if "Set ID" in features:
#     features.remove("Set ID")

# train_x = df_train[features]
# train_y = df_train["target"]

# model.fit(train_x, train_y)

In [13]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# XGBoost classifier의 초기화
model = XGBClassifier(
    objective='binary:logistic',
    random_state=RANDOM_STATE,
    tree_method='gpu_hist',  # GPU 사용 설정
    gpu_id=0
)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'gamma': [0, 0.2],
    'min_child_weight': [1, 5],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [1, 2]
}
features = []

# 데이터프레임의 열을 정수형으로 변환
for col in df_train.columns:
    try:
        df_train[col] = df_train[col].astype(int)
        features.append(col)
    except:
        continue

if "Set ID" in features:
    features.remove("Set ID")

# 목표 변수를 숫자로 매핑
df_train["target"] = df_train["target"].map({'Normal': 0, 'AbNormal': 1})

train_x = df_train[features]
train_y = df_train["target"]

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',  # 평가 지표
    cv=5,                # 교차 검증 fold 수
    verbose=1,          # 상세 출력
    n_jobs=-1           # 모든 CPU 코어 사용
)

# 모델 학습
grid_search.fit(train_x, train_y)

# 최적의 파라미터 및 점수 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:52:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:45: `gpu_id` is deprecated since2.0.0, use `device` instead. E.g. device=cpu/cuda/cuda:0
  warnings.warn(smsg, UserWarning)
c:\Users\ben81\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:52:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Best Parameters: {'colsample_bytree': 1.0, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 50, 'reg_alpha': 0, 'reg_lambda': 1, 'subsample': 0.7}
Best Score: 0.6393617021276595


In [24]:
best_params = grid_search.best_params_
best_model = XGBClassifier(
    objective='binary:logistic',
    random_state=RANDOM_STATE,
    **best_params
)

# 최적의 파라미터로 학습 데이터로 모델 학습
best_model.fit(train_x, train_y)

features = []

for col in df_val.columns:
    try:
        df_val[col] = df_val[col].astype(int)
        features.append(col)
    except:
        continue

if "Set ID" in features:
    features.remove("Set ID")

# Mapping target variable to numerical values
# df_val["target"] = df_val["target"].map({'Normal': 0, 'AbNormal': 1})

val_x = df_val[features]
val_y = df_val["target"]

best_model.predict(val_x)

array([0, 0, 0, ..., 0, 0, 0])

In [15]:
# features = []

# for col in df_val.columns:
#     try:
#         df_val[col] = df_val[col].astype(int)
#         features.append(col)
#     except:
#         continue

# if "Set ID" in features:
#     features.remove("Set ID")

# # Mapping target variable to numerical values
# # df_val["target"] = df_val["target"].map({'Normal': 0, 'AbNormal': 1})

# val_x = df_val[features]
# val_y = df_val["target"]

# model.predict(val_x)

In [25]:
val_pred = best_model.predict(val_x)
val_pred = list(val_pred)
for i in range(len(val_pred)):
    if val_pred[i] == 0:
        val_pred[i] = 'Normal'
    else:
        val_pred[i] = 'AbNormal'

cnt = 0
for a, b in zip(val_y,val_pred):
    if a == b:
        cnt +=1

print(f"{cnt/len(val_y)}")


0.6153191489361702


In [26]:
from collections import Counter

count = Counter(val_pred)
count

Counter({'Normal': 913, 'AbNormal': 262})

## 4. 제출하기

### 테스트 데이터 예측

테스트 데이터 불러오기

In [18]:
df_test_y = pd.read_csv(os.path.join("submission.csv"))

In [19]:
df_test = pd.merge(X, df_test_y, "inner", on="Set ID")
df_test_x = df_test[features]

for col in df_test_x.columns:
    try:
        df_test_x.loc[:, col] = df_test_x[col].astype(int)
    except:
        continue

In [20]:
# test_pred = model.predict(df_test_x)
test_pred = best_model.predict(df_test_x)
test_pred

array([0, 1, 1, ..., 0, 0, 0])

In [21]:
test_pred = list(test_pred)
for i in range(len(test_pred)):
    if test_pred[i] == 0:
        test_pred[i] = 'Normal'
    else:
        test_pred[i] = 'AbNormal'

In [27]:
Counter(test_pred)

Counter({'Normal': 14501, 'AbNormal': 2860})

### 제출 파일 작성

In [23]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**